In [13]:
"""
Complete Image Processing Pipeline:
1. Classify image as FCC or NCC using YOLO model
2. If FCC, convert to NCC using spectral transformation
3. Apply CLAHE + brightness enhancement (only if FCC)
"""

import numpy as np
import imageio
import cv2
import random
from pathlib import Path
from typing import Union, Tuple, Optional
from PIL import Image
from tqdm import tqdm

# Import YOLO functions (assuming they're available)
try:
    from ultralytics import YOLO
    HAS_YOLO = True
except ImportError:
    HAS_YOLO = False
    print("Warning: ultralytics not available. YOLO classification will not work.")

def fcc_to_ncc(
    fcc: np.ndarray,
    in_bit_depth: int = None,
    out_bit_depth: int = 8,
    aB_11bit: float = A_B_11BIT,
    bB: float = B_B,
    cB: float = C_B,
    dB: float = D_B,
    band_idx = (0, 1, 2)
):
    """
    Convert FCC -> NCC using the paper's linear transform.
    
    Parameters:
      - fcc: HxWx3 numpy array (uint8/uint16/float)
      - in_bit_depth: if None, guessed from dtype (best-effort).
      - out_bit_depth: e.g. 8
      - aB_11bit,bB,cB,dB: coefficients from paper (blue band transform)
      - band_idx: a tuple mapping (R_idx, G_idx, B_idx) in input FCC to spectral bands:
          we assume default FCC mapping where:
            R_idx = index for NIR band (FCC's red plane shows NIR)
            G_idx = index for Red band (FCC's green plane shows Red)
            B_idx = index for Green band (FCC's blue plane shows Green)
          Default (0,1,2) corresponds to image read in normal order:
            img[...,0]=R_plane(NIR), img[...,1]=G_plane(Red), img[...,2]=B_plane(Green)
    
    Returns:
      - ncc_rgb: HxWx3 uint8 (or dtype according to out_bit_depth)
    """
    if fcc.ndim != 3 or fcc.shape[2] < 3:
        raise ValueError("fcc must be an H x W x 3 array")
    
    if in_bit_depth is None:
        in_bit_depth = infer_bitdepth_from_dtype(fcc)
    
    # convert to float in native radiometric range [0, 2^in_bits - 1]
    fcc_f = ensure_float_range(fcc, in_bit_depth)
    
    R_idx, G_idx, B_idx = band_idx
    
    # Extract spectral bands from FCC according to assumed mapping:
    # Gf = green spectral band, Rf = red spectral band, IRf = NIR band
    # For the common FCC ordering (R=NIR, G=Red, B=Green):
    IRf = fcc_f[..., R_idx]
    Rf  = fcc_f[..., G_idx]
    Gf  = fcc_f[..., B_idx]
    
    # adjust aB for output bit depth (paper uses 11-bit base)
    aB = adjust_aB_for_bitdepth(aB_11bit, out_bit_depth)
    
    # Compute simulated blue band (Bt) using linear transform:
    Bt = aB + bB * Gf + cB * Rf + dB * IRf
    
    # Clip Bt to valid range for output bit depth:
    out_max = 2**out_bit_depth - 1
    Bt_clipped = np.clip(Bt, 0, out_max)
    
    # For natural color, replace green and red by originals (paper conclusion)
    # Note: R_true = Rf, G_true = Gf, B_true = Bt_clipped
    R_out = np.clip(Rf, 0, out_max)
    G_out = np.clip(Gf, 0, out_max)
    B_out = Bt_clipped
    
    # Compose into RGB array: typical image arrays are in R,G,B order:
    ncc = np.stack([R_out, G_out, B_out], axis=-1)
    
    # convert back to integer type for saving (uint8 or uint16)
    if out_bit_depth <= 8:
        dtype = np.uint8
        ncc_scaled = (ncc).astype(np.uint8)
        # If input was normalized floats, we already scaled earlier
    elif out_bit_depth <= 16:
        dtype = np.uint16
        ncc_scaled = (ncc).astype(np.uint16)
    else:
        # keep float for >16 bits (uncommon)
        dtype = np.float32
        ncc_scaled = ncc.astype(dtype)
    
    return ncc_scaled

def normalize_to_uint8(img: np.ndarray) -> np.ndarray:
    """
    Normalize image to uint8 range [0, 255].
    Handles various input types: uint8, uint16, float.
    """
    if img.dtype == np.uint8:
        return img
    
    # For uint16, normalize to 8-bit
    if img.dtype == np.uint16:
        img_normalized = (img.astype(np.float32) / 65535.0 * 255.0).astype(np.uint8)
        return img_normalized
    
    # For float arrays (assumed 0-1 or 0-max)
    if np.issubdtype(img.dtype, np.floating):
        img_max = img.max()
        if img_max <= 1.0:
            img_normalized = (img * 255.0).astype(np.uint8)
        else:
            img_normalized = (img / img_max * 255.0).astype(np.uint8)
        return img_normalized
    
    # For other integer types, clip and convert
    img_clipped = np.clip(img, 0, 255)
    return img_clipped.astype(np.uint8)

def apply_clahe_and_brightness(
    img: np.ndarray, 
    clip_limit: float = 2.0, 
    tile_grid_size: tuple = (8, 8), 
    brightness_increase: int = 40
) -> np.ndarray:
    """
    Enhances local contrast and detail using CLAHE on uint8 RGB arrays,
    then increases brightness by adding a given value to all channels.

    Parameters:
      - img: uint8 RGB image array (H, W, 3)
      - clip_limit: Contrast limiting threshold (default: 2.0)
      - tile_grid_size: Size of grid for adaptive histogram (default: (8, 8))
      - brightness_increase: Value to add to pixel intensities (default: 40)

    Returns:
      - Enhanced uint8 RGB image with increased brightness
    """
    # Normalize to uint8 if needed
    img_uint8 = normalize_to_uint8(img)
    
    # Ensure 3 channels
    if img_uint8.ndim == 2:
        img_uint8 = np.stack([img_uint8, img_uint8, img_uint8], axis=-1)
    elif img_uint8.ndim == 3 and img_uint8.shape[-1] == 4:
        img_uint8 = img_uint8[..., :3]
    elif img_uint8.ndim == 3 and img_uint8.shape[-1] > 4:
        img_uint8 = img_uint8[..., :3]
    
    # Convert to LAB color space
    lab = cv2.cvtColor(img_uint8, cv2.COLOR_RGB2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab)

    # Apply CLAHE to L channel only
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    l_channel_enhanced = clahe.apply(l_channel)

    # Merge channels and convert back to RGB
    lab_enhanced = cv2.merge([l_channel_enhanced, a_channel, b_channel])
    result = cv2.cvtColor(lab_enhanced, cv2.COLOR_LAB2RGB)

    # Increase brightness by adding brightness_increase, clip to [0, 255]
    brightened = np.clip(result.astype(np.int16) + brightness_increase, 0, 255).astype(np.uint8)

    return brightened

def load_image_any_format(image_path: Union[str, Path]) -> np.ndarray:
    """
    Load image from any format (tif, tiff, png, jpg, jpeg, etc.)
    Handles LZW compression and various bit depths.
    """
    image_path = Path(image_path)
    
    if not image_path.exists():
        raise FileNotFoundError(f"Image file not found: {image_path}")
    
    # Try imageio first (handles LZW compression)
    try:
        img = imageio.imread(str(image_path))
        return img
    except Exception as e1:
        # Fallback to PIL
        try:
            img = Image.open(str(image_path))
            if img.mode != 'RGB':
                img = img.convert('RGB')
            return np.array(img)
        except Exception as e2:
            raise ValueError(f"Could not load image {image_path}: {e1}, {e2}")

def classify_and_convert_image(
    image_path: Union[str, Path],
    model: Optional[YOLO] = None,
    model_path: Union[str, Path] = None,
    threshold: float = 0.880237,
    fcc_class_index: int = None,
    apply_brightness: bool = True,
    brightness_increase: int = 40,
    clip_limit: float = 2.0,
    tile_grid_size: tuple = (8, 8),
    return_diagnostics: bool = True
) -> Union[np.ndarray, Tuple[np.ndarray, dict]]:
    """
    Complete pipeline: Classify image as FCC/NCC, convert if FCC, and apply brightness (only if FCC).
    
    Parameters:
      - image_path: Path to image file (supports .tif, .tiff, .png, .jpg, etc.)
      - model: Pre-loaded YOLO model (if None, will load from model_path)
      - model_path: Path to best.pt model file (required if model is None)
      - threshold: Threshold for FCC classification (default: 0.880237)
      - fcc_class_index: FCC class index in model (if None, auto-detected)
      - apply_brightness: Whether to apply CLAHE + brightness (default: True)
      - brightness_increase: Brightness increase value (default: 40)
      - clip_limit: CLAHE clip limit (default: 2.0)
      - tile_grid_size: CLAHE tile grid size (default: (8, 8))
      - return_diagnostics: If True, returns (image, diagnostics), else just image
    
    Returns:
      - If return_diagnostics=False: Processed image array
      - If return_diagnostics=True: (processed_image, diagnostics_dict)
    
    Diagnostics include:
      - classification: "FCC" or "NCC"
      - converted: Whether FCC->NCC conversion was performed
      - brightness_applied: Whether brightness enhancement was applied
      - fcc_probability: YOLO model's FCC probability
      - original_shape: Original image shape
    """
    if not HAS_YOLO:
        raise ImportError("ultralytics is required. Install with: pip install ultralytics")
    
    # Load model if not provided
    if model is None:
        if model_path is None:
            raise ValueError("Either model or model_path must be provided")
        model_path = Path(model_path)
        if not model_path.exists():
            raise FileNotFoundError(f"Model file not found: {model_path}")
        model = YOLO(str(model_path))
    
    # Auto-detect FCC class index if not provided
    if fcc_class_index is None:
        class_names = model.names
        for idx, name in class_names.items():
            if "FCC" in str(name).upper():
                fcc_class_index = idx
                break
        if fcc_class_index is None:
            # Default to index 1 if 2 classes (alphabetical: NCC=0, FCC=1)
            fcc_class_index = 1
    
    # Load image
    img = load_image_any_format(image_path)
    original_shape = img.shape
    
    # Handle different image shapes - normalize to HWC format
    if img.ndim == 2:
        img = np.stack([img, img, img], axis=-1)
    elif img.ndim == 3:
        if img.shape[0] in [3, 4] and img.shape[-1] not in [3, 4]:
            img = np.transpose(img, (1, 2, 0))
        if img.shape[-1] == 4:
            img = img[..., :3]
        elif img.shape[-1] > 4:
            img = img[..., :3]
    
    # Normalize to uint8 for YOLO classification
    img_for_classification = normalize_to_uint8(img)
    
    # Convert to PIL Image for YOLO (handles better)
    pil_img = Image.fromarray(img_for_classification)
    
    # Classify using YOLO
    results = model(pil_img, verbose=False)
    probs = results[0].probs.data
    fcc_prob = probs[fcc_class_index].item()
    ncc_prob = 1.0 - fcc_prob
    
    # Determine classification
    is_fcc = fcc_prob >= threshold
    classification = "FCC" if is_fcc else "NCC"
    
    # Initialize diagnostics
    diagnostics = {
        "classification": classification,
        "fcc_probability": float(fcc_prob),
        "ncc_probability": float(ncc_prob),
        "threshold_used": threshold,
        "converted": False,
        "brightness_applied": False,
        "original_shape": original_shape
    }
    
    # Convert FCC to NCC if needed, and only apply brightness if FCC
    if is_fcc:
        try:
            # Use fcc_to_ncc function (assumed to be available in this notebook)
            # Infer bit depth from original image
            in_bit_depth = None
            if img.dtype == np.uint16:
                in_bit_depth = 16
            elif img.dtype == np.uint8:
                in_bit_depth = 8
            
            # Convert FCC to NCC
            img = fcc_to_ncc(img, in_bit_depth=in_bit_depth, out_bit_depth=8)
            diagnostics["converted"] = True
        except Exception as e:
            diagnostics["conversion_error"] = str(e)
            print(f"Warning: Could not convert FCC to NCC: {e}")

        # Apply brightness enhancement if requested (only for FCC)
        if apply_brightness:
            try:
                img = apply_clahe_and_brightness(
                    img,
                    clip_limit=clip_limit,
                    tile_grid_size=tile_grid_size,
                    brightness_increase=brightness_increase
                )
                diagnostics["brightness_applied"] = True
            except Exception as e:
                diagnostics["brightness_error"] = str(e)
                print(f"Warning: Could not apply brightness enhancement: {e}")

    # For NCC images, do not apply brightness enhancement
    if return_diagnostics:
        return img, diagnostics
    else:
        return img

def process_folder_classify_and_convert(
    input_folder: Union[str, Path],
    output_folder: Union[str, Path] = None,
    model: Optional[YOLO] = None,
    model_path: Union[str, Path] = None,
    threshold: float = 0.880237,
    apply_brightness: bool = True,
    brightness_increase: int = 40,
    n: int = None,
    random_seed: int = None,
    save_images: bool = True
) -> dict:
    """
    Process all images in a folder: classify, convert FCC->NCC, and apply brightness (only if FCC).
    
    Parameters:
      - input_folder: Path to folder containing images
      - output_folder: Path to save processed images (default: input_folder/processed)
      - model: Pre-loaded YOLO model (if None, will load from model_path)
      - model_path: Path to best.pt model file
      - threshold: Threshold for FCC classification (default: 0.880237)
      - apply_brightness: Whether to apply CLAHE + brightness (default: True)
      - brightness_increase: Brightness increase value (default: 40)
      - n: Number of images to process (if None, processes all)
      - random_seed: Random seed for selecting n images
      - save_images: Whether to save processed images (default: True)
    
    Returns:
      - Dictionary with processing statistics
    """
    input_folder = Path(input_folder)
    if not input_folder.exists():
        raise ValueError(f"Input folder does not exist: {input_folder}")
    
    # Create output folder
    if output_folder is None:
        output_folder = input_folder / "processed"
    else:
        output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)
    
    # Get all image files
    extensions = ['.png', '.jpg', '.jpeg', '.tif', '.tiff', '.bmp']
    image_files = []
    for ext in extensions:
        image_files.extend(list(input_folder.glob(f'*{ext}')))
    
    if len(image_files) == 0:
        raise ValueError(f"No image files found in {input_folder}")
    
    # Select n images if specified
    if n is not None and n < len(image_files):
        if random_seed is not None:
            random.seed(random_seed)
        image_files = random.sample(image_files, n)
    
    # Statistics
    stats = {
        "total": len(image_files),
        "fcc_classified": 0,
        "ncc_classified": 0,
        "converted": 0,
        "brightness_applied": 0,
        "errors": 0
    }
    
    # Process each image
    for img_file in tqdm(image_files):
        try:
            processed_img, diag = classify_and_convert_image(
                img_file,
                model=model,
                model_path=model_path,
                threshold=threshold,
                apply_brightness=apply_brightness,
                brightness_increase=brightness_increase,
                return_diagnostics=True
            )
            
            # Update statistics
            if diag["classification"] == "FCC":
                stats["fcc_classified"] += 1
            else:
                stats["ncc_classified"] += 1
            
            if diag["converted"]:
                stats["converted"] += 1
            if diag["brightness_applied"]:
                stats["brightness_applied"] += 1
            
            # Save processed image
            if save_images:
                output_file = output_folder / img_file.name
                imageio.imwrite(str(output_file), processed_img)
            
            print(f"✓ {img_file.name}: {diag['classification']} "
                  f"(FCC prob: {diag['fcc_probability']:.4f}) "
                  f"{'[Converted]' if diag['converted'] else ''} "
                  f"{'[Brightened]' if diag['brightness_applied'] else ''}")
            
        except Exception as e:
            stats["errors"] += 1
            print(f"✗ Error processing {img_file.name}: {e}")
            continue
    
    # Print summary
    print(f"\n{'='*60}")
    print(f"Processing Summary:")
    print(f"  Total images: {stats['total']}")
    print(f"  FCC classified: {stats['fcc_classified']}")
    print(f"  NCC classified: {stats['ncc_classified']}")
    print(f"  Converted (FCC->NCC): {stats['converted']}")
    print(f"  Brightness applied: {stats['brightness_applied']}")
    print(f"  Errors: {stats['errors']}")
    print(f"  Output folder: {output_folder}")
    print(f"{'='*60}")
    
    return stats

# Example usage:
# # Process a single image
# processed_img, diag = classify_and_convert_image(
#     "path/to/image.tif",
#     model_path="/path/to/best.pt",
#     threshold=0.880237
# )
# print(f"Classification: {diag['classification']}")
# print(f"Converted: {diag['converted']}")
# print(f"Brightness applied: {diag['brightness_applied']}")

In [ ]:
# Process entire folder
stats = process_folder_classify_and_convert(
    input_folder="/kaggle/input/vrsbench-obb/Images_train/Images_train",
    model_path="/kaggle/input/fcc-ncc-yolo/pytorch/default/1/best.pt",
    output_folder = "/kaggle/working/processed",
    threshold=0.880237,
)

  0%|          | 0/20264 [00:00<?, ?it/s]/tmp/ipykernel_47/1126244914.py:191: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(str(image_path))
  0%|          | 2/20264 [00:00<53:50,  6.27it/s]

✓ P2674_0010.png: NCC (FCC prob: 0.0000)  
✓ P0737_0011.png: NCC (FCC prob: 0.0000)  


  0%|          | 4/20264 [00:00<52:33,  6.43it/s]

✓ P0494_0013.png: NCC (FCC prob: 0.0000)  
✓ P1052_0003.png: NCC (FCC prob: 0.0000)  


  0%|          | 6/20264 [00:01<1:00:14,  5.60it/s]

...

 58%|█████▊    | 11653/20264 [30:50<21:41,  6.62it/s]

✓ P0642_0005.png: NCC (FCC prob: 0.0000)  
✓ P1174_0116.png: NCC (FCC prob: 0.0012)  


 58%|█████▊    | 11655/20264 [30:50<22:01,  6.51it/s]

✓ P2397_0002.png: NCC (FCC prob: 0.0000)  
✓ P1114_0017.png: NCC (FCC prob: 0.0082)  


 58%|█████▊    | 11656/20264 [30:50<22:08,  6.48it/s]

In [6]:
"""
fcc_to_ncc.py

Replicates Patra et al.'s spectral-transformation method to create a simulated blue band (NCC)
from a false-colour composite (FCC).

Dependencies: numpy, imageio (or pillow), optionally rasterio for multi-bit geotiffs.
Install: pip install imageio numpy
"""

import numpy as np
import imageio
import os
import matplotlib.pyplot as plt
from pathlib import Path
import random

# Default blue-band coefficients from Table 2 (IKONOS: 11-bit base)
A_B_11BIT = 111.243
B_B = 0.873393
C_B = -0.138254
D_B = -0.100616

def adjust_aB_for_bitdepth(aB_11bit: float, target_bits: int) -> float:
    """
    Eqn (6) in the paper: aB_b = aB_11bit * 2^{-Db}, where Db = 11 - b
    So for b=8, multiply by 2^{-3} = 1/8.
    """
    Db = 11 - target_bits
    return aB_11bit * (2.0 ** (-Db))

def infer_bitdepth_from_dtype(arr: np.ndarray) -> int:
    """Simple inference based on dtype (best-effort)."""
    dt = arr.dtype
    if np.issubdtype(dt, np.integer):
        bits = arr.dtype.itemsize * 8
        # If stored in 16-bit but likely original sensor was 11-bit, we cannot know —
        # we keep dtype bits unless user supplies explicit in_bit_depth param.
        return bits
    else:
        # floats -> assume normalized 0..1 -> treat as 8-bit for display unless user says otherwise
        return 8

def ensure_float_range(arr: np.ndarray, in_bits: int):
    """
    Convert integer array to float in range [0, 2^in_bits - 1].
    If arr is already float in 0..1, scale to bit-range.
    """
    if np.issubdtype(arr.dtype, np.floating):
        # assume 0..1 normalized
        return arr.astype(np.float32) * (2**in_bits - 1)
    else:
        return arr.astype(np.float32)

# --- Helper utilities for FCC detection -------------------------------------

def to_float_radiometric(img: np.ndarray, in_bits: int = None) -> np.ndarray:
    """Return float image in range [0, 2^in_bits - 1]."""
    if in_bits is None:
        in_bits = infer_bitdepth_from_dtype(img)
    if np.issubdtype(img.dtype, np.floating):
        return img.astype(np.float32) * (2**in_bits - 1)
    else:
        return img.astype(np.float32)

# --- Conservative FCC detectors ------------------------------------------

def _vegetation_mean_ratio_B_over_G(img_f: np.ndarray) -> float:
    """Mean(B) / Mean(G). For FCC (B is green spectral), this tends to be > 1."""
    B = img_f[..., 2]
    G = img_f[..., 1]
    # avoid divide-by-zero
    return (B.mean() + 1e-6) / (G.mean() + 1e-6)

def _fraction_B_dominant(img_f: np.ndarray, margin: float = 0.05) -> float:
    """
    Fraction of pixels where B is significantly larger than both R and G.
    margin is in radiometric units (we will normalize so margin is fraction of max).
    """
    R = img_f[..., 0]; G = img_f[..., 1]; B = img_f[..., 2]
    # normalize per-image to 0..1 to make margin interpretable
    mx = img_f.max() if img_f.max() > 0 else 1.0
    Rn, Gn, Bn = R/mx, G/mx, B/mx
    # pixel is B-dominant if B > G*(1+margin) and B > R*(1+margin)
    mask = (Bn > Gn * (1.0 + margin)) & (Bn > Rn * (1.0 + margin))
    return float(mask.mean())

def _ndvi_proxy_fraction(img_f: np.ndarray, threshold: float = 0.15) -> float:
    """
    Proxy NDVI using FCC mapping assumption:
      FCC: R channel = NIR, G channel = Red
    NDVI_proxy = (NIR - Red) / (NIR + Red) = (R - G) / (R + G)
    Fraction of pixels above threshold suggests vegetation visible in NIR => FCC.
    """
    R = img_f[..., 0]; G = img_f[..., 1]
    denom = (R + G)
    # compute safely
    ndvi = np.zeros_like(R, dtype=np.float32)
    nonzero = denom > 1e-6
    ndvi[nonzero] = (R[nonzero] - G[nonzero]) / denom[nonzero]
    return float((ndvi > threshold).mean())

# --- Main conservative decision function ----------------------------------

def is_confident_fcc(
    img,
    in_bit_depth: int = None,
    *,
    # STRICT thresholds (tune down only if you're sure)
    mean_ratio_threshold: float = 1.20,
    frac_B_dom_threshold: float = 0.30,
    ndvi_frac_threshold: float = 0.25,
    debug: bool = False
) -> tuple:
    """
    Return (is_confident_fcc, diagnostics).
    VERY conservative: returns True only if ALL three detectors strongly indicate FCC.
    """
    if img.ndim != 3 or img.shape[2] < 3:
        return False, {"reason": "not a 3-channel image"}

    img_f = to_float_radiometric(img, in_bit_depth)

    # compute detectors
    mean_ratio = _vegetation_mean_ratio_B_over_G(img_f)
    frac_B_dom = _fraction_B_dominant(img_f, margin=0.05)  # 5% margin (relative)
    ndvi_frac = _ndvi_proxy_fraction(img_f, threshold=0.15)

    diagnostics = {
        "mean_ratio_B_over_G": mean_ratio,
        "frac_B_dominant": frac_B_dom,
        "ndvi_proxy_fraction": ndvi_frac,
        "thresholds": {
            "mean_ratio_threshold": mean_ratio_threshold,
            "frac_B_dom_threshold": frac_B_dom_threshold,
            "ndvi_frac_threshold": ndvi_frac_threshold
        }
    }

    # Conservative decision: require ALL tests to pass strongly
    is_fcc = (
        (mean_ratio >= mean_ratio_threshold) and
        (frac_B_dom >= frac_B_dom_threshold) and
        (ndvi_frac >= ndvi_frac_threshold)
    )

    if debug:
        diagnostics["is_fcc"] = is_fcc

    return bool(True), diagnostics



# --- Safe wrapper that uses FCC detection ----------------------

def safe_fcc_to_ncc(
    img,
    in_bit_depth: int = None,
    *,
    require_all_detectors: bool = True,
    debug: bool = False,
    **fcc_to_ncc_kwargs
):
    """
    Calls is_confident_fcc(...) and only runs fcc_to_ncc if there is strong evidence of FCC.
    Returns (output_img, converted_bool, diagnostics).
    
    Parameters:
      - img: HxWx3 array
      - in_bit_depth: Input bit depth (if None, inferred)
      - require_all_detectors: If True, requires all detectors to pass (default: True)
      - debug: If True, includes debug info in diagnostics (default: False)
      - fcc_to_ncc_kwargs: Forwarded to fcc_to_ncc function
    
    Returns:
      - output_img: Converted image (or original if not converted)
      - converted: True if conversion was performed, False otherwise
      - diagnostics: Dictionary with detection results and action taken
    """
    # 1) Quick shape/type checks
    if img.ndim != 3 or img.shape[2] < 3:
        return img, False, {"reason": "not a valid 3-channel image"}

    # 2) Check confidence
    confident, diag = is_confident_fcc(img, in_bit_depth, debug=debug)
    if not confident:
        # If any chance it's NCC -> do not convert (very conservative)
        diag["action"] = "skipped: not confident FCC (to avoid altering possible NCC)"
        return img, False, diag

    # 3) All detectors pass -> do conversion
    try:
        out = fcc_to_ncc(img, in_bit_depth=in_bit_depth, **fcc_to_ncc_kwargs)
    except Exception as e:
        diag["action"] = "conversion_failed"
        diag["error"] = str(e)
        return img, False, diag

    diag["action"] = "converted: confident FCC"
    diag["converted"] = True
    return out, True, diag

In [3]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mkl-fft to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-random to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-umath to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 91.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install numpy==1.26.4 --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 99.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32